In [1]:
import os

os.chdir("..")

print(os.getcwd())

c:\Users\mquick\Documents\zeitgeist blog\projects


In [2]:
import pandas as pd
from words_deeds.types.democracy_indices import electoral, liberal, egalitarian

### Load data

* Vdem
* Words scores dataset
* Get country iso alpha 3 codes using COW codes [here](https://www.jkarreth.net/countrycodes.html)

In [3]:
vdem = pd.read_csv("./words_deeds/data/vdem.csv")

C:\Users\mquick\AppData\Local\Temp\ipykernel_63452\681960494.py:1: DtypeWarning: Columns (364,365,366,399,415,804,836,837,924,1240,1257,1486,3094,3168,3169,3341,3342,3344,3345,3347,3350,3352) have mixed types. Specify dtype option on import or set low_memory=False.
  vdem = pd.read_csv("./words_deeds/data/vdem.csv")


In [4]:
vdem = vdem[["country_text_id", "year", electoral.vdem_index, liberal.vdem_index, egalitarian.vdem_index]]

In [5]:
words_df = pd.read_csv("./words_deeds/data/words_scores.csv")

In [6]:
codes_df = pd.read_csv("https://raw.githubusercontent.com/leops95/cow2iso/master/cow2iso.csv")

In [7]:
codes_df = codes_df[["cow_id", "iso3"]]
codes_df.dropna(subset="cow_id", inplace=True)

In [8]:
codes_df["cow_id"] = codes_df["cow_id"].astype(int)
codes_df.dtypes

cow_id     int32
iso3      object
dtype: object

In [9]:
words_df.dtypes

cowcode          int64
year             int64
egalitarian    float64
electoral      float64
liberal        float64
country         object
words_total    float64
dtype: object

In [10]:
words_df = words_df.merge(codes_df, left_on="cowcode", right_on="cow_id", how="left")\
    .drop(["cow_id", "cowcode"], axis=1)\
    .rename(columns={"cow_id":"iso"})

In [11]:
words_df.loc[words_df["iso3"].isna(), "country"].unique()

array(['Federal Republic of Central America', 'Monaco', 'Liechtenstein',
       'Bavaria', 'Baden', 'Serbia', 'Serbia and Montenegro',
       'Yugoslavia', 'Kosovo', 'Taiwan', 'Vietnam'], dtype=object)

In [12]:
words_df.loc[words_df["country"].isin(["Serbia", "Yugoslavia", "Serbia and Montenegro"]), "iso3"] = "SRB"
words_df.loc[words_df["country"]=="Monaco", "iso3"] = "MCO"
words_df.loc[words_df["country"]=="Liechtenstein", "iso3"] = "LIE"
words_df.loc[words_df["country"]=="Kosovo", "iso3"] = "XKX"
words_df.loc[words_df["country"]=="Taiwan", "iso3"] = "TWN"
words_df.loc[words_df["country"]=="Vietnam", "iso3"] = "VNM"

In [13]:
words_df.dropna(subset=["iso3"], inplace=True)

In [14]:
words_deeds = vdem.merge(words_df, left_on=["country_text_id", "year"], right_on=["iso3", "year"], how="inner")\
    .drop("country_text_id", axis=1)\
    .copy()

In [15]:
country_col = words_deeds.pop("country")
iso_col = words_deeds.pop("iso3")
words_deeds.insert(0, "country", country_col)
words_deeds.insert(1, "iso3", iso_col)

In [16]:
words_deeds.dropna(inplace=True)

In [17]:
words_deeds["deeds_total"] = words_deeds[[electoral.vdem_index, liberal.vdem_index, egalitarian.vdem_index]].mean(axis=1)

In [18]:
for dem_type in [electoral, liberal, egalitarian]:
    words_deeds[f"{dem_type.name}_diff"] = words_deeds[dem_type.vdem_index] - words_deeds[dem_type.name]

In [19]:
words_deeds.to_csv("./words_deeds/data/words_deeds_dataset.csv", index=False)